In [129]:
import random
import pandas as pd
import math

In [130]:
data = pd.read_csv("690103346_T_ONTIME_REPORTING.csv")
data.sample(10)

,DAY_OF_WEEK,OP_CARRIER_AIRLINE_ID,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,CRS_DEP_TIME,DEP_DELAY,Unnamed: 9
132960,2,20304,3567,1163703,31637,1348702,31650,912,80.0,NaN
244306,3,19790,395,1486903,34614,1457002,34570,2041,-1.0,NaN
209769,3,20397,5394,1410005,34100,1199603,31871,1755,-1.0,NaN
173275,3,20363,5381,1195302,31953,1039707,30397,1005,-2.0,NaN
280208,4,20397,5060,1412202,30198,1105703,31057,1015,-3.0,NaN
52262,1,19790,667,1042302,30423,1474703,30559,620,6.0,NaN
149640,2,19790,2982,1501606,31123,1039707,30397,1550,3.0,NaN
192502,3,20363,5078,1058102,30581,1295304,31703,730,31.0,NaN
251180,3,19790,2132,1039707,30397,1449202,34492,1138,-1.0,NaN
225259,3,19393,848,1080003,32575,1477104,32457,1020,186.0,NaN


In [131]:
data['DEP_DELAY'].isna().sum()

16355

In [132]:
print(data.shape)
data = data.loc[~data['DEP_DELAY'].isna()]
data.shape

(583985, 10)


(567630, 10)

In [133]:
print(data.shape)
data = data.drop_duplicates()
data.shape

(567630, 10)


(533272, 10)

In [134]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 533272 entries, 0 to 583983
Data columns (total 10 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   DAY_OF_WEEK            533272 non-null  int64  
 1   OP_CARRIER_AIRLINE_ID  533272 non-null  int64  
 2   OP_CARRIER_FL_NUM      533272 non-null  int64  
 3   ORIGIN_AIRPORT_SEQ_ID  533272 non-null  int64  
 4   ORIGIN_CITY_MARKET_ID  533272 non-null  int64  
 5   DEST_AIRPORT_SEQ_ID    533272 non-null  int64  
 6   DEST_CITY_MARKET_ID    533272 non-null  int64  
 7   CRS_DEP_TIME           533272 non-null  int64  
 8   DEP_DELAY              533272 non-null  float64
 9   Unnamed: 9             0 non-null       float64
dtypes: float64(2), int64(8)
memory usage: 44.8 MB


In [135]:
data.isna().sum(axis=0)

DAY_OF_WEEK                   0
OP_CARRIER_AIRLINE_ID         0
OP_CARRIER_FL_NUM             0
ORIGIN_AIRPORT_SEQ_ID         0
ORIGIN_CITY_MARKET_ID         0
DEST_AIRPORT_SEQ_ID           0
DEST_CITY_MARKET_ID           0
CRS_DEP_TIME                  0
DEP_DELAY                     0
Unnamed: 9               533272
dtype: int64

In [136]:
data = data.iloc[:, :-1]
y = data['DEP_DELAY']
X = data.drop(columns=['DEP_DELAY'])
X = X.values.tolist()

In [137]:
data.head()

,DAY_OF_WEEK,OP_CARRIER_AIRLINE_ID,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,CRS_DEP_TIME,DEP_DELAY
0,1,20363,3280,1039707,30397,1115005,31150,1645,-5.0
1,1,20363,3280,1039707,30397,1115005,31150,1645,-6.0
2,1,20363,3280,1039707,30397,1115005,31150,1645,-4.0
4,1,20363,3280,1115005,31150,1039707,30397,1803,-20.0
5,1,20363,3280,1115005,31150,1039707,30397,1803,-13.0


In [146]:
# TODO сделать 2 варианта умножения матрицы и вектора: по правилам перменожения матриц и умножение числа на вектор
def multiply(G, v):
    result = []
    for i in range(len(G[0])):
        total = 0
        for j in range(len(v)):
            total += v[j] * G[j][i]
        result.append(total)
    return result

In [139]:
#класс для нормировки данных
class scaler():
    def __init__(self):
        self.mean = None
        self.sigma = None
    
    def fit_transform(self, X):
        n, m = len(X), len(X[0])
        if self.mean is None or self.sigma is None:
            self.mean = []
            self.sigma = []
            f = False
        else:
            f = True
        
        for i in range(m):
            mean = 0
            for j in range(n):
                mean += X[j][i]
            if f:
                self.mean[i] = self.mean[i]*0.5 + 0.5*mean/n
            else:
                self.mean.append(mean/n)
        
        for i in range(m):
            sigma = 0
            for j in range(n):
                 sigma += (X[j][i] - self.mean[i])**2
            if f:
                self.sigma[i] = self.sigma[i]*0.5 + 0.5*(sigma/n)**0.5
            else:
                self.sigma.append((sigma/n)**0.5)
        
        for i in range(m):
            for j in range(n):
                X[j][i] = (X[j][i] - self.mean[i])/self.sigma[i]
    
        return X
        
    def transform(self, X):
        assert self.mean or self.sigma 
        n, m = len(X), len(X[0])
        for i in range(m):
            for j in range(n):
                X[j][i] = (X[j][i] - self.mean[i])/self.sigma[i]
        return X
    def get_params(self):
        print(self.mean, self.sigma)

In [140]:
scaler = scaler()
X = scaler.fit_transform(X)

In [163]:
class LinearRegression():
    def __init__(self, r1=0, r2=0):
        self.w = None
        self.r1 = r1
        self.r2 = r2 
        
    def predict(self, X):
        return multiply(X, self.w)
    
    def fit(self, X, y, lr=0.01, max_iter=100):
        pass
    
    def loss(y, y_pred):
        assert len(y) == len(y_true)
        return sum([(i-j)**2 for i, j in zip(y, y_pred)])/len(y)
        
    def get_weights(self):
        return self.w
    
    def get_losses(self):
        return self.losses

In [187]:
#Линейная регрессия sgd
class SGDLinearRegression(LinearRegression):
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    
    def fit(self, X, y, n_sample=1, lr=3e-2, max_iter=100):
        n, m = len(X), len(X[0])
        for i in range(len(X)):
            X[i].append(1)
            
        if self.w is None:
            self.w = []
            for i in range(m+1):
                self.w.append(random.random())
        
        self.losses = []
        for iter_num in range(max_iter):
            y_pred = self.predict(X)
            #self.losses.append(self.loss(y, y_pred))
            grad = self._calc_gradient(X, y, y_pred)
            self.w = [w-lr*gr for w, gr in zip(self.w, grad)]
        
    def _calc_gradient(self, X, y, y_pred):

        grad = list(multiply(X,[i-j for i, j in zip(y_pred, y)]))
#         print(grad)
#         grad = list(map(sum, zip(*grad)))
#         grad = [gr/len(X) for gr in grad]
        return grad

In [188]:
linreg = SGDLinearRegression()
linreg.fit(X, y)

In [191]:
len(linreg.predict(X))

27